In [1]:
import os
import glob
# import csv

from wand.image import Image as wImg
import pandas as pd
import numpy as np

import torch
from torchvision.transforms import v2
from PIL import Image
import pickle

import torchvision.models as models


from torch.utils.data import DataLoader, Dataset

import ast

# from sklearn.decomposition import PCA


# from scipy.spatial.distance import euclidean

# import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Torch device: {device}")

Torch device: cuda


Objetivo desse programa é:


função: ler_svg

ler arquivos .svg, salvar o nome e o path para cada arquivo em um data frame

nome do dataframe vai ser: icons-data-base
coluna dos nome dos arquivos .svg: Nome
coluna com os paths dos arquivos .svg: svg path

------------------------------------------------------------------------------------

função: converter_svg_para_png

transformar os arquivos .svg em .png

através da coluna: svg path, abrirei os arquivos e converterei em .png e salvarei em uma nova pasta, chamada icons-png-database
o nome de cada arquivo .png será escrito de acordo com a coluna: Nome

salvarei o path para os arquivos .png em uma coluna do data frame chamada: png path

--------------------------------------------------------------------------------------

função: converter_png_em_pil_image

converterei os arquivos .png em PIL Image

a função vai ler o path para os arquivos .png através da coluna: png path

salvarei os arquivos PIL Image em uma lista chamada: icons_pil_images

--------------------------------------------------------------------------------------

função: converter_pil_em_tensor

converterei as PIL Image em torch.tensor

a função para converter irá converter as images da lista icons_pil_images

o tensor de cada imagem será salvo em uma nova coluna do data frame: Tensor

--------------------------------------------------------------------------------------

função: analisar_tensor

Irei analisar cada tensor para extrair as caracteristicas de cada imagem

os tensores serão dados atraves da coluna do data frame: Tensor

a função ira retornar as caracteristicas de cada imagem em forma de arrays, 
esses arrays serão salvos em uma nova coluna do data frame chamada: Caracteristicas Array

------------------------------------------------------------------------------------

tenho que ser capas de salvar os arrays e tensores em um arquivo para depois usa-los em um outro programa

In [3]:
def ler_svg(path_to_svgs):

    path_svgs = glob.glob(os.path.join(path_to_svgs, '*.svg'))

    nome_svgs = list()

    # colocando o nome dos arquivos .svg na lista: nome_svgs
    for path in path_svgs:

        nome_n_formatado = path.rsplit(("\\"))[-1]
        nome_formatado = nome_n_formatado.replace(".svg" , "")
        nome_svgs.append(nome_formatado)

    icons_data_base = pd.DataFrame({ 'Nome': nome_svgs, 'svg path': path_svgs})

    return icons_data_base

In [4]:
icons_data_base = ler_svg(r'material-design-icons\svg\filled')

In [5]:
def converter_svg_para_png(data_base, pasta_dos_pngs='icons-png-database'):
    
    if not os.path.exists(pasta_dos_pngs):
        os.makedirs(pasta_dos_pngs)

    png_paths = list()

    for i, row in data_base.iterrows():
        svg_path = row['svg path']
        nome = row['Nome']

        nome_arq_png = f"{nome}.png"
        png_path = os.path.join(pasta_dos_pngs, nome_arq_png)

        with wImg(filename = svg_path) as img:
            img.format = 'png'
            img.save(filename = png_path)
            png_paths.append(png_path)

    data_base['png path'] = png_paths

    return data_base

In [6]:
icons_data_base = converter_svg_para_png(icons_data_base, 'icons-png-database')

In [7]:
def converter_png_em_pil_image(data_base, pasta_dos_pngs='icons-png-database'):
    
    icons_pil_image = list()
    
    for filename in os.listdir(pasta_dos_pngs):
        path = os.path.join(pasta_dos_pngs, filename)
        icon = Image.open(path).convert('RGB')

        icons_pil_image.append(icon)


    data_base['PIL Image'] = icons_pil_image
    
    return data_base


In [8]:
icons_data_base = converter_png_em_pil_image(icons_data_base)

In [9]:
def converter_pil_em_tensor(data_base):
    
    lista_tensores = list()

    img = v2.Compose([
                    v2.Resize((224, 224)),
                    v2.ToImage(), 
                    v2.ToDtype(torch.float32, scale=True),
                    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    
    for pil_image in data_base['PIL Image']:

        img_tensor = img(pil_image)
   
        lista_tensores.append(pickle.dumps(img_tensor).decode('latin1'))
   



    data_base['Tensor'] = lista_tensores


    return data_base


In [10]:
icons_data_base = converter_pil_em_tensor(icons_data_base)

In [ ]:
# def analisar_tensor(data_base, batch_size = 20):

#     torch.cuda.empty_cache()

#     model = models.resnet50(weights = models.ResNet50_Weights.DEFAULT)
#     model = torch.nn.Sequential(*(list(model.children())[:-1]))

#     model.to(device).eval()

#     dataloader = DataLoader(data_base['Tensor'], batch_size=batch_size, shuffle=False)

#     array_caracteristicas = list()

#     for batch in dataloader:
            
#             imagem = batch.to(device)  # Ensure the batch is on the GPU
#             if imagem.dim() == 3:  # Check if input is 3D and add batch dimension
#                 imagem = imagem.unsqueeze(0)

#             with torch.no_grad():
#                 try:
                    
#                     features = model(imagem).squeeze()  # Remove batch and singleton dimensions if necessary
#                     array_caracteristicas.extend(features.cpu().numpy())  # Move back to CPU for numpy conversion

#                 except RuntimeError as e:

#                     print(f"RuntimeError: {e}")
#                     # Reduce batch size if running out of memory
#                     batch_size = max(1, batch_size // 2)
#                     dataloader = DataLoader(data_base['Tensor'], batch_size=batch_size, shuffle=False)
#                     array_caracteristicas = list()

#                     break

#     return np.array(array_caracteristicas)


In [15]:
def analisar_tensores(data_base, batch_size=20):

    def desserializar_tensores(data_base):
        
        tensores_originais = list()

        for tensor_serializado in data_base['Tensor']:
            tensor_original = pickle.loads(tensor_serializado.encode('latin1'))
            tensores_originais.append(tensor_original)

        return tensores_originais    
    
    
    torch.cuda.empty_cache()

    tensores = desserializar_tensores(data_base)

    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    model = torch.nn.Sequential(*(list(model.children())[:-1]))

    model.to(device).eval()


    dataloader = DataLoader(tensores, batch_size=batch_size, shuffle=False)
    
    array_caracteristicas = list()

    while True:
        try:
            for batch in dataloader:

                batch_tensors = [item.clone().detach() for item in batch]
                batch_tensors = torch.stack(batch_tensors).to(device)
                
                if batch_tensors.dim() == 3:
                    batch_tensors = batch_tensors.unsqueeze(0)

                with torch.no_grad():
                    features = model(batch_tensors).squeeze()  
                    array_caracteristicas.extend(features.cpu().numpy())

            break

        except RuntimeError as e:

            print(f"RuntimeError: {e}")
           
            if "out of memory" in str(e):

                batch_size = max(1, batch_size // 2)

                print(f"Reducing batch size to {batch_size} and retrying...")

                dataloader = DataLoader(tensores, batch_size=batch_size, shuffle=False)
                array_caracteristicas = list()

            else:
                raise e

    voltando_bytes = list()      
    for tensor in data_base['Tensor']:
   
        voltando_bytes.append(pickle.dumps(tensor).decode('latin1'))
    
    data_base['Tensor'] = voltando_bytes
            
    data_base['Array características'] = array_caracteristicas[:len(data_base)]

    data_base['Array características'] = data_base['Array características'].apply(lambda x: np.array2string(x, separator=','))

    return data_base

In [16]:
icons_data_base = analisar_tensores(icons_data_base)

In [17]:
def salvar_dataframe(data_base, nome_arquivo_csv):
    data_base.to_csv(nome_arquivo_csv, index=False)

In [ ]:
salvar_dataframe(icons_data_base, 'new-icon-db.csv')

In [ ]:
def ler_dataframe(nome_arquivo_csv = "icons-data-base.csv"):

    nome_data_base = pd.read_csv(nome_arquivo_csv)
    nome_data_base['Array características'] = nome_data_base['Array características'].apply(lambda x: np.array(ast.literal_eval(x)))
    nome_data_base['Tensor'] = nome_data_base['Tensor'].apply(lambda x: x.encode('latin1'))

    return nome_data_base


In [ ]:
new_icon_db = ler_dataframe('new-icon-db.csv')

In [18]:
def main(path_to_svgs, pasta_dos_pngs = 'icons-png-database', batch_size = 20, nome_arquivo_csv = 'icons-data-frame.csv'):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Torch device: {device}")

    data_base = ler_svg(path_to_svgs)

    data_base = converter_svg_para_png(data_base, pasta_dos_pngs)

    data_base = converter_png_em_pil_image(data_base, pasta_dos_pngs)

    data_base = converter_pil_em_tensor(data_base)

    data_base = analisar_tensores(data_base, analisar_tensores(data_base, batch_size))

    salvar_dataframe(data_base, nome_arquivo_csv)


In [ ]:
main(r'material-design-icons\svg\filled')